# Test KServe Integration

This example notebook is loosely based on [this](https://github.com/kubeflow/examples/blob/master/kserve/sdk/first_isvc_kserve.ipynb) upstream example. Note that this only works with `Serverless` deployment mode at the moment.

- create Inference Service
- perform inference

## Setup

In [ ]:
# Please check the requirements.in file for more details
!pip install -r requirements.txt

### Import required packages

In [ ]:
import requests

from kserve import (
    constants,
    KServeClient,
    V1beta1InferenceService,
    V1beta1InferenceServiceSpec,
    V1beta1PredictorSpec,
    V1beta1SKLearnSpec,
)
from kubernetes.client import V1ObjectMeta
from tenacity import retry, stop_after_attempt, wait_exponential

## Define Inference Service

In [ ]:
ISVC_NAME = "sklearn-iris"

In [ ]:
isvc = V1beta1InferenceService(
    api_version=constants.KSERVE_V1BETA1,
    kind=constants.KSERVE_KIND_INFERENCESERVICE,
    metadata=V1ObjectMeta(
        name=ISVC_NAME,
        annotations={"sidecar.istio.io/inject": "false"},
    ),
    spec=V1beta1InferenceServiceSpec(
        predictor=V1beta1PredictorSpec(
            sklearn=V1beta1SKLearnSpec(
                storage_uri="gs://kfserving-examples/models/sklearn/1.0/model"
            )
        )
    ),
)

## Create Inference Service

### Initialise KServe Client

In [ ]:
client = KServeClient()

### Submit Inference Service

In [ ]:
client.create(isvc)

In [ ]:
@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(30),
    reraise=True,
)
def assert_isvc_created(client, isvc_name):
    """Wait for the Inference Service to be created successfully."""
    assert client.is_isvc_ready(ISVC_NAME), f"Failed to create Inference Service {isvc_name}."

In [ ]:
assert_isvc_created(client, ISVC_NAME)

## Perform Inference

In [ ]:
@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(30),
    reraise=True,
)
def get_isvc_url(client, isvc_name):
    """Get ISVC from client and return it's URL

    Add a retry clause to address the following intermittent issue
    https://github.com/canonical/bundle-kubeflow/issues/1100"""
    isvc_object = client.get(isvc_name)
    return isvc_object["status"]["address"]["url"]


url = get_isvc_url(client, ISVC_NAME)
print("Inference URL:", url)

Hit the service for predictions using the above URL.

In [ ]:
inference_input = {"instances": [[6.8, 2.8, 4.8, 1.4], [6.0, 3.4, 4.5, 1.6]]}
response = requests.post(f"{url}/v1/models/sklearn-iris:predict", json=inference_input)
print(response.text)

In [ ]:
res = response.json()
# verify that the predictions are as expected
assert res.get("predictions"), "Failed to get predictions!"
assert res["predictions"] == [1, 1], "Predictions different than expected!"

## Delete Inference Service

In [ ]:
client.delete(ISVC_NAME);

In [ ]:
@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(30),
    reraise=True,
)
def assert_isvc_deleted(client, isvc_name):
    """Wait for the Inference Service to be deleted."""
    try:
        # try fetching the ISVC to verify it was deleted successfully
        isvc = client.get(isvc_name)
        assert not isvc, f"Failed to delete Inference Service {isvc_name}!"
    except RuntimeError as err:
        assert "Not Found" in str(err), f"Caught unexpected exception: {err}"

In [ ]:
assert_isvc_deleted(client, ISVC_NAME)